# AGE CSV Support

In [5]:
import age 
import psycopg2
from age.age_csv import *
import random

conn = psycopg2.connect(
    host="localhost",
    port="5430",
    dbname="postgresDB",
    user="postgresUser",
    password="postgresPW")
GRAPH_NAME = 'bitnine_global_inic'


### Create a Random GRAPH with 2000 nodes and 4000 edges in AGE

In [6]:
number_of_nodes = 2000
number_of_edges = 2000

age.setUpAge(conn, GRAPH_NAME)
age.deleteGraph(conn, GRAPH_NAME)
age.setUpAge(conn, GRAPH_NAME)


try:
    query = ""
    for i in range(number_of_nodes):
        query += """
        SELECT * from cypher('%s', $$ CREATE (n:Person {id : '%s'} ) $$) as (v agtype); """ % (GRAPH_NAME, str(i))
        
    with conn.cursor() as cursor:
        cursor.execute(query)
        query=""
        conn.commit()
    
    for i in range(number_of_edges):
        a = 0
        b = 0
        while(a==b):
            a = random_number = random.randint(0, number_of_nodes-1)
            b = random_number = random.randint(0, number_of_nodes-1)
        
        query += """
        SELECT * from cypher('%s', 
        $$ 
            MATCH (a:Person {id: '%s'}), (b:Person {id: '%s'}) 
            CREATE (a)-[r:workWith ]->(b)
        $$) as (v agtype); """ % (GRAPH_NAME,str(a), str(b))

    with conn.cursor() as cursor:
        cursor.execute(query)
        conn.commit()
        
        cursor.execute("""
        SELECT count(*) from cypher( '%s', $$ MATCH (n) RETURN n$$) as (n agtype);
        """ % (GRAPH_NAME))
        for row in cursor: print(row[0] , "nodes created")
        cursor.execute("""
        SELECT count(*) from cypher( '%s', $$ MATCH p=(a )-[b]->(c) RETURN p $$) as (n agtype);
        """ % (GRAPH_NAME))
        for row in cursor: print(row[0] , "edges created")
except Exception as e:
    print(e)


2000 nodes created
2000 edges created


### Save to csv
- Change the `conn` and `GRAPH_NAME` according to your age configuration

In [7]:
import os
# print(os.getcwd())
save_csv(connection=conn,
         GRAPH_NAME=GRAPH_NAME, directory= os.getcwd())

Created a directory
Directory name : bitnine_global_inic_data
Directory path : /home/moontasir/Desktop/networkx/age/drivers/python/samples/bitnine_global_inic_data


### LOAD from csv
- Change the `conn` according to your age configuration
- Give `GRAPH_NAME` where you want to load
- Give `directory` where you have stored the csv file
- You must follow the naming rules for generating nodes and edges
- Vertices file name format : `label_name.csv`
- Edges file name format : `edge_label_name.csv`
- Example:
    - Creating nodes with label People : `People.csv`
    - Creating edges with label Purchase : `edge_Purchase.csv`

In [8]:
age.setUpAge(conn, GRAPH_NAME)
age.deleteGraph(conn, GRAPH_NAME)
age.setUpAge(conn, GRAPH_NAME)
data = load_csv(connection=conn,
                GRAPH_NAME=GRAPH_NAME,
                # directory=os.path.join(os.path.dirname(__file__), 'data')
                directory="./bitnine_global_inic_data"
                )

-------------------------------------------
Checking file...
-------------------------------------------
2 files are ok
Found vertices file : 1
Found edge file : 1 
-------------------------------------------
Loaded 1000 vertices with label : Person within 1.361075067999991 s
Loaded 2000 vertices with label : Person within 2.68238522700085 s
Loaded 1000 edges with label : workWith within 10.059133271000974 s
Loaded 2000 edges with label : workWith within 17.566719402000672 s
